# Module Imports

In [1]:
## IMPORT MODULES
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy.interpolate import griddata
import scipy.odr


In [2]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

# Dataframe

In [3]:
## FULL DATAFRAME
df_raw = pd.read_csv("Wide-Long_2-Policy_Summary.csv")

## DROP EXTRANEOUS DATAPOINTS
# df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vz_d']>=2.5)].index)
# df_raw = df_raw.drop(df_raw[(df_raw['landing_rate']<=0.15) & (df_raw['vx_d']>=1.0)].index)
# df_raw = df_raw.dropna()
# df_raw = df_raw.drop(df_raw[df_raw['My_d']<-7.7].index)




## GROUP RAW DATA BY IC AND FILTER BY MAX LANDING RATE
# df_raw_max = df_raw.groupby(['vel_IC','phi_IC']).agg({'landing_rate_4_leg':"max"}).reset_index()

idx = df_raw.groupby(['vel_IC','phi_IC'])['landing_rate_4_leg'].transform(max) == df_raw['landing_rate_4_leg']
df_max = df_raw[idx].reset_index()


# ## CLEAN RAW DATA
# df_max['My_d'] = df_max['My_d'].apply(lambda x: np.abs(x)) # Convert My to magnitude
# df_max['My_d'] = df_max['My_d'].apply(lambda x: 7.7 if x>7.7 else x) # Cap My_d to max possible moment




# Landing Rate Data

In [4]:
## LANDING RATE DATA (RAW)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_raw['vx']
Y = df_raw['vz']
Z = df_raw['landing_rate_4_leg']

# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('Vel_x (m/s)')
ax.set_ylabel('Vel_z (m/s)')
ax.set_zlabel('4-Leg_Landing Rate (%)')
ax.set_title('Policy Landing Rate')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
## LANDING RATE DATA (Best Policy)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx']
Y = df_max['vz']
Z = df_max['landing_rate_4_leg']


# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('Vel_x (m/s)')
ax.set_ylabel('Vel_z (m/s)')
ax.set_zlabel('4-Leg_Landing Rate (%)')
ax.set_title('Landing Rate (Best Policy)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx']
Y = df_max['vz']
Z = df_max['landing_rate_4_leg']

# DEFINE COLOR FORMATS AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)

# CREATE PLOTS AND COLOR BAR
ax.plot_trisurf(X,Y,Z,cmap = cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")


# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Landing Rate (%)')
ax.set_title('Landing Rate (Best Policy)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
## Max LANDING RATE (SMOOTHED SURFACE)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx']
Y = df_max['vz']
Z = df_max['landing_rate_4_leg']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//7))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//7))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
# surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")

# PLOT LIMITS AND INFO
ax.set_zlim(0,1)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Landing Rate (%)')
ax.set_title('Smoothed Landing Rate (Best Policy)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-7-fe9db3873d8e>:21: UserWarning: Z contains NaN values. This may result in rendering artifacts.
  surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)


# RREV vs IC

In [8]:
## RREV DATA (RAW)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_raw['vx']
Y = df_raw['vz']

# X = df_raw['vel_IC']
# Y = df_raw['phi_IC']
Z = df_raw['RREV_threshold']

# CREATE PLOTS AND COLOR BAR
ax.scatter(X,Y,Z)


# PLOT LIMITS AND INFO
# ax.set_zlim(0,1)

ax.set_xlabel('Vel_x (m/s)')
ax.set_ylabel('Vel_z (m/s)')
ax.set_zlabel('RREV_threshold (1/s)')
ax.set_title('Policy RREV (Raw)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
## LANDING RATE DATA (Best Policy)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx']
Y = df_max['vz']
Z = df_max['RREV_threshold']
C = df_max['landing_rate_4_leg']


# CREATE PLOTS AND COLOR BAR
# DEFINE COLOR FORMATS AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm,zorder=1)


# PLOT LIMITS AND INFO
ax.set_xlabel('Vel_x (m/s)')
ax.set_ylabel('Vel_z (m/s)')
ax.set_zlabel('RREV_threshold (1/s)')
ax.set_title('Policy RREV (Max Landing Rate)')
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx']
Y = df_max['vz']
Z = df_max['RREV_threshold']



# DEFINE COLOR FORMATS AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=3,vmax=7)

# CREATE PLOTS AND COLOR BAR
ax.plot_trisurf(X,Y,Z,cmap = cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")


# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('RREV_threshold (1/s)')
ax.set_title('Policy RREV (Max Landing Rate)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
## Max LANDING RATE (SMOOTHED SURFACE)
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# DEFINE VARIABLES
X = df_max['vx']
Y = df_max['vz']
Z = df_max['RREV_threshold']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//7))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//7))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=3,vmax=7)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
# surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV threshold")

# PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('RREV_threshold (1/s)')
ax.set_title('Policy RREV (Max Landing Rate)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-11-6f053a21f9e4>:21: UserWarning: Z contains NaN values. This may result in rendering artifacts.
  surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)


In [12]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate_4_leg>={0.4}")


# DEFINE VARIABLES
X = df_temp['vx']
Y = df_temp['vz']
Z = df_temp['RREV_threshold']

# SOMETHING ABOUT DEFINING A GRID
xi = np.linspace(X.min(),X.max(),(len(Z)//3))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//3))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

# DEFINE COLOR FORMAT AND LIMITS
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=3.5,vmax=5.5)

# CREATE PLOTS AND COLOR BAR
surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")

# PLOT LIMITS AND INFO
ax.set_xlim(0,4)
ax.set_ylim(0,4)
ax.set_zlim(3,6)

ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('RREV_threshold (1/s)')

ax.set_title('Optimal RREV_thr vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-12-b52b1b14537d>:24: UserWarning: Z contains NaN values. This may result in rendering artifacts.
  surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm,edgecolors='grey',linewidth=0.25)


# Rotation Time Data

# My_d vs IC Data

# Impact Angle Data

In [13]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_max.query(f"landing_rate_4_leg>={0.4}")


# DEFINE VARIABLES
X = df_temp['impact_vx_mean']
Y = df_temp['impact_vz_mean']
Z = df_temp['impact_eul_mean']
C = df_temp['landing_rate_4_leg']


# # CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# # PLOT LIMITS AND INFO
ax.set_xlabel('X-Velocity (m/s)')
ax.set_ylabel('Z-Velocity (m/s)')
ax.set_zlabel('Impact Angle (deg)')

ax.set_title('Impact Angle vs IC (Optimal Data)')

ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180,210])



# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Moment vs RREV

In [14]:
fig = plt.figure()
ax = fig.add_subplot(111)

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['My_d']
C = df_temp['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.Greys
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV_threshold (1/s)')
ax.set_ylabel('My_d (N*mm)')
ax.set_title('Policy: My_d vs RREV_thr')


ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.1,'Motors Maxed Out \n(My_d = 7.7 N*mm)')




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

UndefinedVariableError: name 'landing_rate' is not defined

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

# CLEAN DATA
df_temp = df_max.query(f"landing_rate>={0.4}")

# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['My_d']
C = df_temp['vz_d']

# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=1.5,vmax=4)
ax.scatter(X,Y,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Z-Velocity (m/s)")



# PLOT LIMITS AND INFO
ax.set_xlabel('RREV_threshold (1/s)')
ax.set_ylabel('My_d (N*mm)')
ax.set_title('Policy: My_d vs RREV_thr')




ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.1,'Motors Maxed Out \n(My_d = 7.7 N*mm)')




plt.show()

# POLICY FITTING

## RAW POLICY RELATION

In [20]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_temp = df_raw.query(f"landing_rate_4_leg>={0.6}")



# DEFINE VARIABLES
X = df_temp['RREV_threshold']
Y = df_temp['OF_y_flip_mean']
Z = df_temp['My_d']
C = df_temp['landing_rate_4_leg']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
# ax.set_xlim(6,2)
# ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Raw Policy Relation')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# LINEAR MODEL FIT

## Regression with y-based loss

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_reg = df_max.query(f"landing_rate>={0.6}")


# LINEAR REGRESSION
X_reg = df_reg[['RREV_threshold','OF_y']]
Y_reg = df_reg['My_d']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: My_d = {intercept:.2f} + {coef[0]:.2f}*RREV + {coef[1]:.2f}*OF_y \n")
print(f"R^2 = {R2:.3f}")



# DEFINE VARIABLES
X = df_reg['RREV_threshold']
Y = df_reg['OF_y']
Z = df_reg['My_d']
C = df_reg['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.plot_trisurf(X_reg['RREV_threshold'],X_reg['OF_y'],ypred,alpha=0.75,label='Linear_Regression',zorder=1) # Surface plot
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm) # Scatter plot
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Optimal Policy Relation \n(Z-Based Error)')



plt.show()

In [ ]:
df_cov = df_reg[['RREV_threshold','OF_y','My_d']]
df_cov.cov()

In [ ]:
df_reg.query(f"My_d<={3.5}").sort_values('OF_y')

## Regression with x-based loss


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_reg = df_max.query(f"landing_rate>={0.6}")


# LINEAR REGRESSION
X_reg = df_reg[['My_d','OF_y']]
Y_reg = df_reg['RREV_threshold']

ypred, intercept,coef,R2 = linreg(X_reg,Y_reg)
print(f"Equation: RREV_thr = {intercept:.2f} + {coef[0]:.2f}*My_d + {coef[1]:.2f}*OF_y \n")
# print(f"Equation: My_d = ((RREV_thr - {intercept:.2f}) - {coef[1]:.2f}*OF_y)/{coef[0]:.2f})")
print(f"R^2 = {R2:.3f}")



# DEFINE VARIABLES
X = df_reg['My_d']
Y = df_reg['OF_y']
Z = df_reg['RREV_threshold']
C = df_reg['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.plot_trisurf(X_reg['My_d'],X_reg['OF_y'],ypred,alpha=0.75,label='Linear_Regression',zorder=1) # Surface plot
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm) # Scatter plot
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")



# PLOT LIMITS AND INFO
ax.set_xlim(10,0)
ax.set_ylim(0,-10)
ax.set_zlim(3,6)



ax.set_xlabel('My_d (N*mm)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('RREV (1/s)')

ax.set_title('Optimal Policy Relation \n(X-Based Error)')



plt.show()




# Regression w/ Orthogonal Distance

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

# CLEAN DATA
df_reg = df_max.query(f"landing_rate>={0.6}")


# LINEAR REGRESSION
def linfit(beta, x):
    return beta[0]*x[0] + beta[1]*x[1] + beta[2] 

x1 = df_reg['RREV_threshold']
x2 = df_reg['OF_y']
y = df_reg['My_d']
x = np.row_stack( (x1, x2) ) #odr doesn't seem to work with column_stack

linmod = scipy.odr.Model(linfit)
data = scipy.odr.Data(x, y)
odrfit = scipy.odr.ODR(data, linmod, beta0=[1., 1., 1.])
odrres = odrfit.run()
odrres.pprint()


y_reg = odrres.beta[0]*x1 + odrres.beta[1]*x2 + odrres.beta[2]

print(f"Equation: My_d = {odrres.beta[2]:.2f} + {odrres.beta[0]:.2f}*RREV + {odrres.beta[1]:.2f}*OF_y \n")
# print(f"R^2 = {R2:.3f}")



# DEFINE VARIABLES
X = df_max['RREV_threshold']
Y = df_max['OF_y']
Z = df_max['My_d']
C = df_max['landing_rate']


# CREATE PLOTS AND COLOR BAR
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
ax.plot_trisurf(x1,x2,y_reg,alpha=0.75,label='Linear_Regression',zorder=1) # Surface plot
ax.scatter(X,Y,Z,c=C,cmap=cmap,norm=norm) # Scatter plot
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing Rate (%)")



# PLOT LIMITS AND INFO
ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV (1/s)')
ax.set_ylabel('OF_y (rad/s)')
ax.set_zlabel('My_d (N*mm)')

ax.set_title('Optimal Policy Relation \n(Orthogonal Regression)')



plt.show()

In [ ]:
import numpy as np
import scipy.odr

n = 1000
t = np.linspace(0, 1, n)

def linfit(beta, x):
    return beta[0]*x[0] + beta[1]*x[1] + beta[2] #notice changed indices for x

x1 = 2.5*np.sin(2*np.pi*6*t)+4
x2 = 0.5*np.sin(2*np.pi*7*t + np.pi/3)+2

x = np.row_stack( (x1, x2) ) #odr doesn't seem to work with column_stack

e = 0.25*np.random.randn(n)
y = 3*x[0] + 4*x[1] + 5 + e #indices changed

linmod = scipy.odr.Model(linfit)
data = scipy.odr.Data(x, y)
odrfit = scipy.odr.ODR(data, linmod, beta0=[1., 1., 1.])
odrres = odrfit.run()
odrres.pprint()


In [ ]:
odrres.beta

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
y_reg = odrres.beta[0]*x1 + odrres.beta[1]*x2 + odrres.beta[2]

ax.scatter(x1,x2,y)
ax.plot_trisurf(x1,x2,y_reg)
